In [ ]:
# vocab.py
import random
import numpy as np
'''
    A default vocab implementation and base class, to provide random letters and numbers.
'''
class Vocab():
    def __init__(self):
        self.vocab = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.size = len(self.vocab)
        indices = range(self.size)
        self.index = dict(zip(self.vocab, indices))
    # return random string by given length
    def rand_string(self, length):
        # if len(vocab) == 0 raise exception
        return "".join(random.sample(self.vocab, length))
    # get symbol (char in vocabulary) by its ordinal
    def get_sym(self, idx):
        # if idx >= len(self.vocab) raise exception
        return self.vocab[idx]
    # given a symbol, return its ordinal in given vocabulary.
    def get_index(self, sym):
        return self.index[sym]
    # given 'abc', return [10, 11, 12]
    def to_indices(self, text):
        return [self.index[c] for c in text]
    # given [10, 11, 12], return 'abc'
    def to_text(self, indices):
        return "".join([self.vocab[i] for i in indices])
    # given '01', return vector [1 0 0 0 0 0 0 0 0 0 ... 0 \n 0 1 0 0 0 0 ... 0]
    def text_to_one_hot(self, text):
        num_labels = np.array(self.to_indices(text))
        n = len(text)
        categorical = np.zeros((n, self.size))
        categorical[np.arange(n), num_labels] = 1
        return categorical.ravel()
    # translate one hot vector to text.
    def one_hot_to_text(self, onehots):
        text_len = onehots.shape[0] // self.size
        onehots = np.reshape(onehots, (text_len, self.size))
        indices = np.argmax(onehots, axis = 1)
        return self.to_text(indices)
    
#if __name__ == "__main__":
#    # test code
#    vocab = Vocab()
#    print(vocab.rand_string(4))
#    print(vocab.get_sym(10))
#    print(vocab.get_index('a'))
#    print(vocab.size)
#    print(vocab.text_to_one_hot("abc"))

In [ ]:
from PIL import Image, ImageFont, ImageDraw, ImageFilter
import random

class Captcha:
    '''
    size: width, height in pixel
    font: font family(string), size (unit pound) and font color (in "#rrggbb" format)
    bgcolor: in "#rrggbb" format
    '''
    def __init__(self, size, font, bgcolor, length = 4):
        #todo: add param check and transform here
        self.width, self.height = size
        self.font_family, self.font_size, self.font_color = font
        #self.bgcolor = bgcolor
        self.bgcolor = "#000000"
        self.len = length
        self.vocab = Vocab()
        self.font = ImageFont.truetype(self.font_family, self.font_size)
    def get_text(self):
        return self.vocab.rand_string(self.len)
    # by default, draw center align text
    def draw_text(self, str):
        dr = ImageDraw.Draw(self.im)
        font_width, font_height = self.font.getsize(str)
        # don't know why, but for center align, I should divide it by 2, other than 3
        dr.text(((self.width - font_width) / 2, (self.height - font_height) / 2), str, fill = self.font_color, font = self.font)
    def draw_background(self):
        pass
    def transform(self):
        params = [1 - float(random.randint(1, 2)) / 100,
                  0,
                  0,
                  0,
                  1 - float(random.randint(1, 10)) / 100,
                  float(random.randint(1, 2)) / 500,
                  0.001,
                  float(random.randint(1, 2)) / 500
                  ]
        self.im = self.im.transform((self.width, self.height), Image.PERSPECTIVE, params)
    def filter(self):
        self.im.filter(ImageFilter.EDGE_ENHANCE_MORE)
    # by default, add no noises
    def add_noise(self):
        pass
    def get_captcha(self):
        self.im = Image.new("RGB", (self.width, self.height), (self.bgcolor))
        self.draw_background()
        str = self.get_text()
        self.draw_text(str)
        self.add_noise()
        #self.transform()
        self.filter()
        return self.im, str
class JrttCaptcha(Captcha):
    def __init__(self, size = (120, 40), font = ("./font/arial.ttf", 25, "#FFFFFF"), bgcolor = (255, 255, 255), dot_rate = 0.075):
        Captcha.__init__(self, size, font, bgcolor)
        self.dot_rate = dot_rate
    def add_noise(self):
        dr = ImageDraw.Draw(self.im)
#        # add lines
#        nb_lines = random.randint(1, 2)
#        for i in range(nb_lines):
#            # 避免begin和end太靠近，导致生成的干扰线太短
#            begin = (random.randint(0, self.width)/2, random.randint(0, self.height)/2)
#            end = (random.randint(self.width / 2, self.width), random.randint(0, self.height))
#            #dr.line([begin, end], fill = (0, 0, 0))
            
        # add dots
        for w in range(self.width):
            for h in range(self.height):
                if random.randint(0, 100) / 100 <= self.dot_rate:
                    dr.point((w, h), fill = (255, 255, 255))
    def draw_text(self, str):
        display_text = [" "] * (len(str) * 2 - 1)
        for i in range(len(str)):
            display_text[i * 2] = str[i]
        super().draw_text(str)


In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*
from captcha.image import ImageCaptcha
from PIL import Image
import numpy as np
import random
import string
class generateCaptcha():
    def __init__(self,
                 width = 120,#验证码图片的宽
                 height = 40,#验证码图片的高
                 char_num = 4,#验证码字符个数
                 characters = string.digits + string.ascii_uppercase):#验证码组成，数字+大写字母+小写字母
        self.width = width
        self.height = height
        self.char_num = char_num
        self.characters = characters
        self.classes = len(characters)
    def gen_captcha(self,batch_size = 50):
        X = np.zeros([batch_size,self.height,self.width,1])
        img = np.zeros((self.height,self.width),dtype=np.uint8)
        Y = np.zeros([batch_size,self.char_num,self.classes])
        #image = ImageCaptcha(width = self.width,height = self.height)
        while True:
            for i in range(batch_size):
                #captcha_str = ''.join(random.sample(self.characters,self.char_num))
                #img = image.generate_image(captcha_str).convert('L')
                
                img, captcha_str = JrttCaptcha().get_captcha()
                img = img.convert('L')
                clearNoise(img, 125, 3, 6) 
                
                img = np.array(img.getdata())
                X[i] = np.reshape(img,[self.height,self.width,1])/255.0
                for j,ch in enumerate(captcha_str):
                    Y[i,j,self.characters.find(ch)] = 1
            Y = np.reshape(Y,(batch_size,self.char_num*self.classes))
            yield X,Y
    def decode_captcha(self,y):
        y = np.reshape(y,(len(y),self.char_num,self.classes))
        return ''.join(self.characters[x] for x in np.argmax(y,axis = 2)[0,:])
    def get_parameter(self):
        return self.width,self.height,self.char_num,self.characters,self.classes
    def gen_test_captcha(self):
        #image = ImageCaptcha(width = self.width,height = self.height)
        #captcha_str = ''.join(random.sample(self.characters,self.char_num))
        #img = image.generate_image(captcha_str)
        
        img, captcha_str = JrttCaptcha().get_captcha()
        img = img.convert('L')
        clearNoise(img, 125, 3, 6) 
        img.save('./test_captcha/' + captcha_str + '.jpg')

In [ ]:
generateCaptcha().gen_test_captcha()

In [ ]:
cap = JrttCaptcha()
#每调用一次，生成一个<图像,文本>对。其中图象可看成是输入，而文本可以看成是真值
img, text = cap.get_captcha()
img.save(text + ".jpg")
print(text)